# parsing results

In [36]:
import pandas as pd

In [37]:
results_dir = "/home/jpt/git/mta_simulator_redo/data_analysis/results/stationing/results"
logs_dir = "/home/jpt/git/mta_simulator_redo/data_analysis/results/stationing/logs"

In [38]:
def find_line_no_of_string(file_path, search_string):
    # Initialize a variable to store the line number
    line_number = None

    # Open the file and search for the string
    with open(file_path, "r") as file:
        lines = file.readlines()
        for i, line in enumerate(lines, start=1):
            if search_string in line:
                line_number = i
                break  # Exit the loop once the first matching line is found

    if line_number is not None:
        return line_number
    return None

In [39]:
import os

bus_string = "bus_id,dwell_time,agg_delay,service_time,total_served,service_kms,current_stop,status,type,total_deadsecs,starting_stop"
stop_string = "stop_id,total_passenger_ons,total_passenger_offs,total_passenger_walk_away"
end_string = "passenger waiting dict list"


# Function to recursively traverse directories and read files
def traverse_directories(root_dir):
    buses_df = []
    stops_df = []
    for root, dirs, files in os.walk(root_dir):
        for file_name in files:
            # Check if the file meets your criteria, e.g., has a specific extension
            # if "5MININTERVAL" in root:
            #     continue
            if file_name.endswith(".csv"):
                file_path = os.path.join(root, file_name)
                with open(file_path, "r") as file:
                    # Read the contents of the file
                    file_contents = file.read()
                # Process the file contents as needed
                # print(f"Reading file: {file_path}")
                # print(file_contents)
                bus_line_no = find_line_no_of_string(file_path, bus_string)
                stop_line_no = find_line_no_of_string(file_path, stop_string)
                end_line = find_line_no_of_string(file_path, end_string)
                if bus_line_no and stop_line_no and end_line:
                    # print(bus_line_no, stop_line_no)

                    # Initialize a list to store the lines between A and B
                    lines_between_A_and_B = []

                    # Read the file and capture the lines between A and B
                    with open(file_path, "r") as file:
                        lines = file.readlines()
                        lines_between_A_and_B = lines[bus_line_no - 1 : stop_line_no - 1]

                    # Define the output CSV file
                    output_bus_csv_path = f"{root}/bus.csv"
                    output_stop_csv_path = f"{root}/stop.csv"

                    # Write the captured lines to a CSV file
                    with open(output_bus_csv_path, "w") as csv_file:
                        csv_file.writelines(lines_between_A_and_B)

                    # Initialize a list to store the lines between B and C
                    lines_between_B_and_C = []

                    # Read the file and capture the lines between B and C
                    with open(file_path, "r") as file:
                        lines = file.readlines()
                        lines_between_B_and_C = lines[stop_line_no - 1 : end_line - 1]

                    # Write the captured lines to a CSV file
                    with open(output_stop_csv_path, "w") as csv_file:
                        csv_file.writelines(lines_between_B_and_C)

                    bus_df = pd.read_csv(output_bus_csv_path)
                    config = root.split("/")[-1]
                    bus_df["date"] = config.split("_")[0]
                    if "AGENCY" in root:
                        bus_df["method"] = config.split("_")[-1]
                    elif "SEARCH" in root:
                        bus_df["method"] = "SEARCH"
                    elif "5MININTERVAL" in root:
                        bus_df["method"] = "MCTS_5"
                    else:
                        bus_df["method"] = config.split("_")[11]
                    bus_df["real_env"] = " ".join(config.split("_")[5:7])
                    bus_df["mcts_env"] = " ".join(config.split("_")[7:9])
                    bus_df["noise"] = int(config.split("_")[9])
                    bus_df.to_csv(output_bus_csv_path)

                    stop_df = pd.read_csv(output_stop_csv_path)
                    stop_df["date"] = config.split("_")[0]
                    if "AGENCY" in root:
                        stop_df["method"] = config.split("_")[-1]
                    elif "SEARCH" in root:
                        stop_df["method"] = "SEARCH"
                    elif "5MININTERVAL" in root:
                        stop_df["method"] = "MCTS_5"
                    else:
                        stop_df["method"] = config.split("_")[11]
                    stop_df["real_env"] = " ".join(config.split("_")[5:7])
                    stop_df["mcts_env"] = " ".join(config.split("_")[7:9])
                    stop_df["noise"] = int(config.split("_")[9])
                    stop_df.to_csv(output_stop_csv_path)

                    buses_df.append(bus_df)
                    stops_df.append(stop_df)

    bus_df = pd.concat(buses_df)
    stop_df = pd.concat(stops_df)
    return bus_df, stop_df


# Call the function with your root directory
bus_df, stop_df = traverse_directories(results_dir)

In [40]:
bus_df.query("method == 'MCTS_5'")

,bus_id,dwell_time,agg_delay,service_time,total_served,service_kms,current_stop,status,type,total_deadsecs,starting_stop,date,method,real_env,mcts_env,noise
0,125,23995.9,2.10,58320.0,22.0,0.0,GXIRIVEF,1,BusType.REGULAR,0.0,MCC,20210130,MCTS_5,REAL WORLD,TEST WORLD,10
1,140,22346.2,0.80,53220.0,8.0,0.0,MCC,1,BusType.REGULAR,0.0,HICHICNN,20210130,MCTS_5,REAL WORLD,TEST WORLD,10
2,1803,16131.0,0.02,35880.0,10.0,0.0,WHICHASF,1,BusType.REGULAR,0.0,MCC,20210130,MCTS_5,REAL WORLD,TEST WORLD,10
3,1806,24911.8,0.05,56760.0,6.0,0.0,MCC,1,BusType.REGULAR,0.0,WHICHASF,20210130,MCTS_5,REAL WORLD,TEST WORLD,10
4,1809,23468.9,0.35,48960.0,13.0,0.0,EZECONWN,1,BusType.REGULAR,0.0,EZECONWN,20210130,MCTS_5,REAL WORLD,TEST WORLD,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,724,16130.1,76.51,57704.8,14.0,0.0,100OAKS,1,BusType.REGULAR,0.0,POWPOWNM,20210302,MCTS_5,REAL WORLD,TEST WORLD,5
68,725,2379.0,160.54,24349.4,26.0,0.0,19SSHESN,1,BusType.REGULAR,0.0,MCC,20210302,MCTS_5,REAL WORLD,TEST WORLD,5
69,730,16511.5,6.52,39240.0,14.0,0.0,33AJOHNM,1,BusType.REGULAR,0.0,LPSCI,20210302,MCTS_5,REAL WORLD,TEST WORLD,5
70,733,2953.2,151.95,28260.0,18.0,0.0,MCC,1,BusType.REGULAR,0.0,MCC,20210302,MCTS_5,REAL WORLD,TEST WORLD,5


In [41]:
stop_df.method.unique()

array(['MCTS', 'AGENCY', 'MCTS_5', 'SEARCH', 'BASELINE'], dtype=object)

In [42]:
bus_df.query("date == '20210312' and noise == 0 and real_env == 'REAL WORLD' and mcts_env == 'TEST WORLD'").groupby(
    "method"
).agg({"total_served": "sum"})
bus_df.query(
    "date == '20210312' and noise == 0 and real_env == 'REAL WORLD' and mcts_env == 'TEST WORLD' and type == 'BusType.OVERLOAD'"
).groupby("method").agg({"service_kms": "sum", "service_time": "sum"})

,service_kms,service_time
method,,
AGENCY,103.38,155428.0
BASELINE,114.19,154613.1
MCTS,50.29,196176.7
SEARCH,101.64,155503.2


In [43]:
grouped_stop_df = (
    stop_df.groupby(["date", "method", "real_env", "mcts_env", "noise"])
    .agg({"total_passenger_ons": "sum", "total_passenger_offs": "sum", "total_passenger_walk_away": "sum"})
    .reset_index()
)

In [44]:
dates = grouped_stop_df.date.unique().tolist()
dates

['20210130',
 '20210202',
 '20210211',
 '20210302',
 '20210312',
 '20210401',
 '20210425',
 '20210606',
 '20210624',
 '20210725',
 '20210826',
 '20210829',
 '20210901',
 '20210928']

In [45]:
date = "20210130"
grouped_stop_df.query("date == @date")

,date,method,real_env,mcts_env,noise,total_passenger_ons,total_passenger_offs,total_passenger_walk_away
0,20210130,AGENCY,REAL WORLD,TEST WORLD,0,464.0,464.0,0.0
1,20210130,AGENCY,REAL WORLD,TEST WORLD,1,464.0,464.0,0.0
2,20210130,AGENCY,REAL WORLD,TEST WORLD,5,464.0,464.0,0.0
3,20210130,AGENCY,REAL WORLD,TEST WORLD,10,464.0,464.0,0.0
4,20210130,AGENCY,TEST WORLD,TEST WORLD,0,2492.0,2492.0,0.0
5,20210130,BASELINE,REAL WORLD,TEST WORLD,0,464.0,464.0,0.0
6,20210130,BASELINE,REAL WORLD,TEST WORLD,1,464.0,464.0,0.0
7,20210130,BASELINE,REAL WORLD,TEST WORLD,5,464.0,464.0,0.0
8,20210130,BASELINE,REAL WORLD,TEST WORLD,10,464.0,464.0,0.0
9,20210130,BASELINE,TEST WORLD,TEST WORLD,0,2492.0,2492.0,0.0


In [46]:
# bus_df.query("date == @date and noise == 0 and real_env == 'REAL WORLD' and mcts_env == 'TEST WORLD'").groupby(
#     "method"
# ).agg({"total_served": "sum"})
bus_df.query(
    "date == @date and noise == 0 and real_env == 'REAL WORLD' and mcts_env == 'TEST WORLD' and type == 'BusType.OVERLOAD'"
).groupby("method").agg({"service_kms": "sum", "service_time": "sum"})

,service_kms,service_time
method,,
AGENCY,63.15,47335.5
BASELINE,63.15,46850.0
MCTS,10.76,57960.0
SEARCH,63.78,48253.0


# plotting service kms (real world env)

In [47]:
bus_df.query("status == 0")

,bus_id,dwell_time,agg_delay,service_time,total_served,service_kms,current_stop,status,type,total_deadsecs,starting_stop,date,method,real_env,mcts_env,noise
2,1311,19185.0,1.09,39600.0,10.0,0.0,25ACLASN,0,BusType.REGULAR,0.0,MTA,20210312,MCTS,REAL WORLD,TEST WORLD,0
10,1803,19975.8,0.00,60594.0,14.0,0.0,21AACKSF,0,BusType.REGULAR,0.0,MCC,20210312,MCTS,REAL WORLD,TEST WORLD,0
22,1821,11272.2,6.87,28002.0,11.0,0.0,5AVJRGSN,0,BusType.REGULAR,0.0,MCC,20210312,MCTS,REAL WORLD,TEST WORLD,0
27,1827,9555.7,0.21,23224.0,19.0,0.0,NXOMCNLY,0,BusType.REGULAR,0.0,MCC,20210312,MCTS,REAL WORLD,TEST WORLD,0
37,1913,0.0,0.00,0.0,0.0,0.0,GXIMADLB,0,BusType.REGULAR,0.0,GXIMADLB,20210312,MCTS,REAL WORLD,TEST WORLD,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2019,213.5,0.00,0.0,0.0,0.0,MCC,0,BusType.REGULAR,0.0,MCC,20210401,BASELINE,REAL WORLD,TEST WORLD,1
0,120,11635.4,2.11,28890.0,9.0,0.0,GALCARSN,0,BusType.REGULAR,0.0,GXIRIVEF,20210425,AGENCY,REAL WORLD,TEST WORLD,1
21,197,11856.1,1.68,34347.0,10.0,0.0,GALCONNN,0,BusType.REGULAR,0.0,GXIRIVEF,20210425,AGENCY,REAL WORLD,TEST WORLD,1
28,2013,6439.7,8.30,19838.0,10.0,0.0,CLAWHASN,0,BusType.REGULAR,0.0,MCC,20210425,AGENCY,REAL WORLD,TEST WORLD,1


In [48]:
MCTS_DATES = [
    "20210130",
    "20210211",
    "20210302",
    "20210312",
    "20210401",
    "20210425",
    "20210606",
    "20210624",
    "20210725",
    "20210829",
    "20210901",
]

In [49]:
MCTS_TYPE = "MCTS_5"
real_env = "TEST WORLD"
mcts_env = "TEST WORLD"
a = (
    stop_df.query("noise == 0 and real_env == @real_env and mcts_env == 'TEST WORLD'")
    .groupby(["date", "method"])
    .agg({"total_passenger_ons": "sum", "total_passenger_offs": "sum", "total_passenger_walk_away": "sum"})
)
b = (
    bus_df.query("noise == 0 and real_env == @real_env and mcts_env == @mcts_env and type == 'BusType.OVERLOAD'")
    .groupby(["date", "method"])
    .agg({"total_served": "sum", "service_kms": "sum", "service_time": "sum"})
)
df = pd.merge(a, b, on=["date", "method"])
df = df.reset_index()
df["all_pax"] = df["total_passenger_ons"] + df["total_passenger_walk_away"]

df = df[df["date"].isin(MCTS_DATES)]
df = df[df["method"].isin(["AGENCY", "BASELINE", "SEARCH", MCTS_TYPE])]
# df.to_csv("./results/total_ons.csv")
display(df.head())

# df['normalized_service_kms'] = df.groupby(level='date')['service_kms'].apply(lambda x: x / x.max()).reset_index(level=0, drop=True)
df = df.reset_index()
df = df.groupby("method").agg({"service_kms": list}).reset_index()

import numpy as np

df["std"] = df["service_kms"].apply(lambda x: np.std(x))
df["mean"] = df["service_kms"].apply(lambda x: np.mean(x))
df = df.reset_index()
df = df[df["method"].isin(["AGENCY", "BASELINE", "SEARCH", MCTS_TYPE])]
df.head()

,date,method,total_passenger_ons,total_passenger_offs,total_passenger_walk_away,total_served,service_kms,service_time,all_pax
0,20210130,AGENCY,2492.0,2492.0,0.0,85.0,132.85,134980.7,2492.0
1,20210130,BASELINE,2492.0,2492.0,0.0,85.0,133.60,136060.7,2492.0
3,20210130,SEARCH,2492.0,2492.0,0.0,85.0,110.86,134860.7,2492.0
8,20210211,AGENCY,3766.0,3766.0,157.0,103.0,201.61,266547.6,3923.0
9,20210211,BASELINE,3766.0,3766.0,157.0,103.0,216.38,266673.9,3923.0


,index,method,service_kms,std,mean
0,0,AGENCY,"[132.85, 201.61, 140.41, 287.36, 262.39, 153.0...",87.780137,198.724545
1,1,BASELINE,"[133.6, 216.38, 153.09, 296.87, 264.91, 153.87...",88.548893,204.841818
2,2,MCTS_5,"[167.64, 125.31, 196.92000000000002, 171.32, 5...",65.225345,157.875714
3,3,SEARCH,"[110.86, 195.6, 134.97, 284.9, 259.2, 155.97, ...",88.232617,195.343636


In [50]:
import plotly.express as px

df.loc[1, "method"] = "GARAGE"
sort_dict = {"GARAGE": 0, "AGENCY": 1, "SEARCH": 2, "MCTS": 3, "MCTS_5": 4}
df = df.iloc[df["method"].map(sort_dict).sort_values().index]

fig = px.line(df, x="method", y="mean", error_y="std", markers=True, title="Line Plot with Error Bars")
fig.update_traces(line={"width": 5})
fig.data[0].error_y.thickness = 5
fig.update_xaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_yaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_layout(
    width=800,
    height=500,
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent paper background
    margin=dict(l=5, r=5, t=5, b=5),
    title="",
    font=dict(
        # family="Courier New, monospace",
        size=18,
        color="black",
    ),
)

fig.update_xaxes(showline=True, linewidth=1, linecolor="black", mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor="black", mirror=True)  # Change x and y-axis labels
fig.update_xaxes(title_text="Stationing Plan")
fig.update_yaxes(title_text="Deadhead kilometers")
# fig.update_xaxes(categoryorder='array', categoryarray= ['GARAGE', 'AGENCY', 'SEARCH', 'MCTS', 'MCTS_5'])
# fig.write_image("./plots/new_deadhead.pdf")
fig.show()

In [51]:
df = pd.read_csv("/home/jpt/git/mta_simulator_redo/data_analysis/results/total_ons.csv", index_col=0)
df = df.reset_index()
df = df.groupby("method").agg({"service_kms": list}).reset_index()
df["std"] = df["service_kms"].apply(lambda x: np.std(x))
df["std"] = df["std"] / np.sqrt(len(MCTS_DATES))
df["mean"] = df["service_kms"].apply(lambda x: np.mean(x))
df = df.reset_index()
df = df.drop("service_kms", axis=1)
a = df[df["method"].isin(["AGENCY", "BASELINE", "SEARCH", "MCTS"])]
a["type"] = "service_kms"

In [79]:
df = pd.read_csv("/home/jpt/git/mta_simulator_redo/data_analysis/results/total_ons.csv", index_col=0)
df = df.reset_index()
df = df.groupby("method").agg({"total_served": list}).reset_index()
df["std"] = df["total_served"].apply(lambda x: np.std(x))
df["std"] = df["std"] / np.sqrt(len(MCTS_DATES))
df["mean"] = df["total_served"].apply(lambda x: np.mean(x))
df = df.reset_index()
df = df.drop("total_served", axis=1)
b = df[df["method"].isin(["AGENCY", "BASELINE", "SEARCH", "MCTS"])]
b["type"] = "total_served"
df = pd.concat([b, a])
df

,index,method,std,mean,type
0,0,AGENCY,25.694793,164.090909,total_served
1,1,BASELINE,25.694793,164.090909,total_served
2,2,MCTS,27.195483,171.090909,total_served
3,3,SEARCH,25.694793,164.090909,total_served
0,0,AGENCY,26.466707,198.724545,service_kms
1,1,BASELINE,26.698496,204.841818,service_kms
2,2,MCTS,24.627822,142.323636,service_kms
3,3,SEARCH,26.603135,195.343636,service_kms


In [80]:
import plotly.express as px

df["method"] = pd.Categorical(df["method"], ["BASELINE", "AGENCY", "SEARCH", "MCTS"])
df = df.sort_values("method")
df = df.query("type == 'total_served'")
# df.loc[1, "method"] = "GARAGE"
# sort_dict = {"GARAGE": 0, "AGENCY": 1, "SEARCH": 2, "MCTS": 3, "MCTS_5": 4}
# df = df.iloc[df["method"].map(sort_dict).sort_values().index]
display(df)
fig = px.line(
    df,
    x="method",
    y="mean",
    error_y="std",
    markers=True,
    title="Line Plot with Error Bars"
    # , facet_col="type"
)
fig.update_traces(line={"width": 5})
fig.data[0].error_y.thickness = 5
# fig.data[1].error_y.thickness = 5
fig.update_xaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_yaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_layout(
    width=600,
    height=500,
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent paper background
    margin=dict(l=5, r=5, t=5, b=5),
    title="",
    font=dict(
        # family="Courier New, monospace",
        size=18,
        color="black",
    ),
)

fig.update_xaxes(showline=True, linewidth=1, linecolor="black", mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor="black", mirror=True)  # Change x and y-axis labels
# fig.update_xaxes(title_text="Stationing and Dispatch Plan")
fig.update_yaxes(title_text="Mean passengers served<br>by substitute buses")


fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# turn off axis titels of all axes
fig.for_each_xaxis(lambda x: x.update({"title": ""}))
# fig.for_each_yaxis(lambda y: y.update({'title': ''}))
fig.add_annotation(
    showarrow=False, xanchor="center", xref="paper", x=0.5, yref="paper", y=-0.15, text="Stationing Plans"
)

# fig.update_xaxes(categoryorder='array', categoryarray= ['GARAGE', 'AGENCY', 'SEARCH', 'MCTS', 'MCTS_5'])
fig.write_image("./plots/new_served.png", scale=3)
fig.show()

,index,method,std,mean,type
1,1,BASELINE,25.694793,164.090909,total_served
0,0,AGENCY,25.694793,164.090909,total_served
3,3,SEARCH,25.694793,164.090909,total_served
2,2,MCTS,27.195483,171.090909,total_served


In [82]:
import plotly.express as px

df = pd.read_csv("/home/jpt/git/mta_simulator_redo/data_analysis/results/total_ons.csv", index_col=0)
df = df.reset_index()
df = df.groupby("method").agg({"total_served": list}).reset_index()
df["std"] = df["total_served"].apply(lambda x: np.std(x))
df["std"] = df["std"] / np.sqrt(len(MCTS_DATES))
df["mean"] = df["total_served"].apply(lambda x: np.mean(x))
df = df.reset_index()
df = df.drop("total_served", axis=1)
b = df[df["method"].isin(["AGENCY", "BASELINE", "SEARCH", "MCTS"])]
b["type"] = "total_served"
df = pd.concat([b, a])
df["method"] = pd.Categorical(df["method"], ["BASELINE", "AGENCY", "SEARCH", "MCTS"])
df = df.sort_values("method")
df = df.query("type == 'service_kms'")
# df.loc[1, "method"] = "GARAGE"
# sort_dict = {"GARAGE": 0, "AGENCY": 1, "SEARCH": 2, "MCTS": 3, "MCTS_5": 4}
# df = df.iloc[df["method"].map(sort_dict).sort_values().index]
display(df)
fig = px.line(
    df,
    x="method",
    y="mean",
    error_y="std",
    markers=True,
    title="Line Plot with Error Bars"
    # , facet_col="type"
)
fig.update_traces(line={"width": 5})
fig.data[0].error_y.thickness = 5
# fig.data[1].error_y.thickness = 5
fig.update_xaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_yaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_layout(
    width=600,
    height=500,
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent paper background
    margin=dict(l=5, r=5, t=5, b=5),
    title="",
    font=dict(
        # family="Courier New, monospace",
        size=18,
        color="black",
    ),
)

fig.update_xaxes(showline=True, linewidth=1, linecolor="black", mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor="black", mirror=True)  # Change x and y-axis labels
# fig.update_xaxes(title_text="Stationing and Dispatch Plan")
fig.update_yaxes(title_text="Mean total deadhead miles<br>for all substitute buses")


fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# turn off axis titels of all axes
fig.for_each_xaxis(lambda x: x.update({"title": ""}))
# fig.for_each_yaxis(lambda y: y.update({'title': ''}))
fig.add_annotation(
    showarrow=False, xanchor="center", xref="paper", x=0.5, yref="paper", y=-0.15, text="Stationing Plans"
)

# fig.update_xaxes(categoryorder='array', categoryarray= ['GARAGE', 'AGENCY', 'SEARCH', 'MCTS', 'MCTS_5'])
fig.write_image("./plots/new_deadhead.png", scale=3)

fig.show()

,index,method,std,mean,type
1,1,BASELINE,26.698496,204.841818,service_kms
0,0,AGENCY,26.466707,198.724545,service_kms
3,3,SEARCH,26.603135,195.343636,service_kms
2,2,MCTS,24.627822,142.323636,service_kms


In [223]:
df = pd.read_parquet(
    "/home/jpt/git/mta_simulator_redo/code_root/scenarios/TEST_WORLD/20200307_noise_1/sampled_ons_offs_dict_20200307.parquet"
)
df.offs.sum()

12722.0

In [224]:
df = pd.read_parquet(
    "/home/jpt/git/mta_simulator_redo/code_root/scenarios/TEST_WORLD/20200307_noise_1/chains/ons_offs_dict_chain_20200307_2.parquet"
)
df.offs.sum()

12614.0

In [225]:
df = pd.read_parquet(
    "/home/jpt/git/mta_simulator_redo/code_root/scenarios/TEST_WORLD/20200307_noise_10/sampled_ons_offs_dict_20200307.parquet"
)
df.offs.sum()

16050.0